# Spark Machine Learning Pipeline - Task 2 (Big Data Module)

Students:  
Miguel Esteras & Alberto Ruiz Benitez de Lugo

This coursework presents an implementation and application of Spark Machine Learning Pipelines. Evaluating them regarding preprocessing, parametrisation, and scaling.

## Section A) Choice of dataset and task (20%)

### Santander products <-- DataSet

We have selected "Santander Products" as our dataset. The reason of this choice has been the large amount of predictors and responses available within this data. This amount of possible variable combinations increases the complexity of the problem and therefore the selected model should has a deep level of abstraction when the amount of data is huge. 
Furthermore, this is a good example about why big data implementations, such as Spark, is worth to implement.

Therefore, the high level of complexity of the data has been the main driver to choose "Random Forest" as our model for this pipeline. This method is currently state-of-the-art in many different Machine Learning Field, like computer vision. This model shows a very good performance in both, classification and regression implementations. Moreover, it presents a good level of abstraction, as commented above, that may provide good results with this complex and large dataset.

### Task

The goal (task) of this pipeline is to predict whether a financial product will be purchased by a consumer or not. Thus, given all the data available we are going to create a Random Forest model to predict when a particular Santander Product (we need to select which one) will be purchased by a customer or not. 

The method applied is estimating which combination of predictors (age, renta, ...etc) are more likely to select a particular nest of products. Then, we use this information to predict whether a particular customer would purchase a particular product or not. We need to select this product in advance, in our case we have selected "ind_ctju_fin_ult1" as our target product.



## Section B) Machine Learning Pipeline in Spark (25%)

## 1. Data set initial analysis and summary of pipeline task

### 1.1 Summary of Pipeline

- Load Data and first preprocessing (1.2)
- Descriptive statistics (1.3)
- Data Cleaning (1.4)
- Machine learning pipeline Implementation using Random Forest (2.)


### 1.2. Loading data to RDD and first preprocessing

In [1]:
# load dependencies
import numpy as np
import pandas as pd

type_dict = {'ncodpers':np.int32,
            'ind_ahor_fin_ult1':np.uint8, 'ind_aval_fin_ult1':np.uint8, 
            'ind_cco_fin_ult1':np.uint8,'ind_cder_fin_ult1':np.uint8,
            'ind_cno_fin_ult1':np.uint8,'ind_ctju_fin_ult1':np.uint8,'ind_ctma_fin_ult1':np.uint8,
            'ind_ctop_fin_ult1':np.uint8,'ind_ctpp_fin_ult1':np.uint8,'ind_deco_fin_ult1':np.uint8,
            'ind_deme_fin_ult1':np.uint8,'ind_dela_fin_ult1':np.uint8,'ind_ecue_fin_ult1':np.uint8,
            'ind_fond_fin_ult1':np.uint8,'ind_hip_fin_ult1':np.uint8,'ind_plan_fin_ult1':np.uint8,
            'ind_pres_fin_ult1':np.uint8,'ind_reca_fin_ult1':np.uint8,'ind_tjcr_fin_ult1':np.uint8,
            'ind_valo_fin_ult1':np.uint8,'ind_viv_fin_ult1':np.uint8, 'ind_recibo_ult1':np.uint8 }

# load data from server into dataframe (only loading the top 100,000 for demonstration purpose)
df = pd.read_csv("/data/tempstore/santander-products/train_ver2.csv",
                 nrows = 100000,
                 dtype = type_dict)


### 1.3. Descriptive Statistics

In [2]:
df.describe()

,ncodpers,ind_nuevo,indrel,indrel_1mes,conyuemp,tipodom,cod_prov,ind_actividad_cliente,renta,ind_ahor_fin_ult1,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
count,1.000000e+05,99317.000000,99317.000000,99317.000000,0.0,99317.0,99231.000000,99317.000000,8.171600e+04,100000.0,...,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,99790.000000,99790.000000,100000.000000
mean,1.040513e+06,0.000211,1.126303,1.000060,NaN,1.0,24.944775,0.410302,1.155589e+05,0.0,...,0.000110,0.001260,0.000200,0.019250,0.018640,0.004630,0.000060,0.032939,0.035485,0.097760
std,6.282306e+04,0.014540,3.515940,0.010992,NaN,0.0,13.646314,0.491891,1.594097e+05,0.0,...,0.010488,0.035474,0.014141,0.137403,0.135251,0.067887,0.007746,0.178478,0.185002,0.296991
min,8.885090e+05,0.000000,1.000000,1.000000,NaN,1.0,1.000000,0.000000,2.539800e+03,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.019914e+06,0.000000,1.000000,1.000000,NaN,1.0,11.000000,0.000000,6.217089e+04,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.055276e+06,0.000000,1.000000,1.000000,NaN,1.0,28.000000,0.000000,8.961021e+04,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.088296e+06,0.000000,1.000000,1.000000,NaN,1.0,36.000000,1.000000,1.330239e+05,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.375586e+06,1.000000,99.000000,3.000000,NaN,1.0,52.000000,1.000000,2.425324e+07,0.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### 1.4. Data Cleaning

In [3]:
# keep only unique id
unique_ids = pd.Series(df["ncodpers"].unique())
df = df[df.ncodpers.isin(unique_ids)]  
df.count() # number of instances

fecha_dato               100000
ncodpers                 100000
ind_empleado              99317
pais_residencia           99317
sexo                      99317
age                      100000
fecha_alta                99317
ind_nuevo                 99317
antiguedad               100000
indrel                    99317
ult_fec_cli_1t              128
indrel_1mes               99317
tiprel_1mes               99317
indresi                   99317
indext                    99317
conyuemp                      0
canal_entrada             99312
indfall                   99317
tipodom                   99317
cod_prov                  99231
nomprov                   99231
ind_actividad_cliente     99317
renta                     81716
segmento                  99309
ind_ahor_fin_ult1        100000
ind_aval_fin_ult1        100000
ind_cco_fin_ult1         100000
ind_cder_fin_ult1        100000
ind_cno_fin_ult1         100000
ind_ctju_fin_ult1        100000
ind_ctma_fin_ult1        100000
ind_ctop

In [4]:
# eliminate mostly empty columns and redundant variables
df.drop(["tipodom","cod_prov", "ult_fec_cli_1t","conyuemp"],axis=1,inplace=True)

In [5]:
# transform to numeric and set missing values to nan
df['age']=pd.to_numeric(df.age, errors='coerce')
df['ind_nuevo']=pd.to_numeric(df.ind_nuevo, errors='coerce')
df['antiguedad']=pd.to_numeric(df.antiguedad, errors='coerce')
df['indrel']=pd.to_numeric(df.indrel, errors='coerce')
df['renta']=pd.to_numeric(df.renta, errors='coerce')
df['indrel_1mes']=pd.to_numeric(df.indrel_1mes, errors='coerce')

In [6]:
# Remove age outliers and nan from age variable
df.loc[df.age < 18,"age"]  = df.loc[(df.age >= 18) & (df.age <= 30),"age"].mean(skipna=True) # replace outlier con mean
df.loc[df.age > 100,"age"] = df.loc[(df.age >= 30) & (df.age <= 100),"age"].mean(skipna=True) # replace outlier con mean
df["age"].fillna(df["age"].mean(),inplace=True) # replace nan with mean
df["age"] = df["age"].astype(int)

In [7]:
# transfor dates to datetime datatype
df["fecha_dato"] = pd.to_datetime(df["fecha_dato"],format="%Y-%m-%d")
df["fecha_alta"] = pd.to_datetime(df["fecha_alta"],format="%Y-%m-%d")
df["fecha_dato"].unique()

array(['2015-01-28T00:00:00.000000000'], dtype='datetime64[ns]')

In [8]:
# fill datetime missing values
dates=df.loc[:,"fecha_alta"].sort_values().reset_index()
median_date = int(np.median(dates.index.values))
df.loc[df.fecha_alta.isnull(),"fecha_alta"] = dates.loc[median_date,"fecha_alta"] 

In [9]:
# check all missing values
df.isnull().any()

fecha_dato               False
ncodpers                 False
ind_empleado              True
pais_residencia           True
sexo                      True
age                      False
fecha_alta               False
ind_nuevo                 True
antiguedad                True
indrel                    True
indrel_1mes               True
tiprel_1mes               True
indresi                   True
indext                    True
canal_entrada             True
indfall                   True
nomprov                   True
ind_actividad_cliente     True
renta                     True
segmento                  True
ind_ahor_fin_ult1        False
ind_aval_fin_ult1        False
ind_cco_fin_ult1         False
ind_cder_fin_ult1        False
ind_cno_fin_ult1         False
ind_ctju_fin_ult1        False
ind_ctma_fin_ult1        False
ind_ctop_fin_ult1        False
ind_ctpp_fin_ult1        False
ind_deco_fin_ult1        False
ind_deme_fin_ult1        False
ind_dela_fin_ult1        False
ind_ecue

In [10]:
# Replace missing values in target features with 0
# target features = boolean indicator as to whether or not that product was owned that month
df.loc[df.ind_nomina_ult1.isnull(), "ind_nomina_ult1"] = 0
df.loc[df.ind_nom_pens_ult1.isnull(), "ind_nom_pens_ult1"] = 0

In [11]:
# Replace other missing values
df.loc[df["ind_nuevo"].isnull(),"ind_nuevo"] = 1                   # new customers id '1'
df.loc[df.antiguedad.isnull(),"antiguedad"] = df.antiguedad.min()
df.loc[df.antiguedad <0, "antiguedad"] = 0                         # new customer antiguedad '0'
df.loc[df.indrel.isnull(),"indrel"] = 1 
df.loc[df.ind_actividad_cliente.isnull(),"ind_actividad_cliente"] = \
df["ind_actividad_cliente"].median()                   # fill in customer activity missing
df.loc[df.nomprov.isnull(),"nomprov"] = "UNKNOWN"      # known values for city of residence
df.loc[df.indfall.isnull(),"indfall"] = "N"            # missing deceased index set to N
df.loc[df.tiprel_1mes.isnull(),"tiprel_1mes"] = "A"    # customer status, if missing = active 
df.tiprel_1mes = df.tiprel_1mes.astype("category")     # customer status as categorical

In [12]:
# Customer type normalization as categorical variable 
map_dict = { 1.0:"1", "1.0":"1", "1":"1", "3.0":"3", "P":"P", 3.0:"3", 2.0:"2", "3":"3", "2.0":"2", "4.0":"4", "4":"4", "2":"2"}
df.indrel_1mes.fillna("P",inplace=True)
df.indrel_1mes = df.indrel_1mes.apply(lambda x: map_dict.get(x,x))
df.indrel_1mes = df.indrel_1mes.astype("category")

In [13]:
# remove rows with any nan value left
df = df.dropna(subset=['renta', 'segmento', 'canal_entrada', 'ind_empleado', 
                       'pais_residencia', 'indresi', 'indresi', 'sexo'], how='any')

In [14]:
# check all missing values are gone
df.isnull().any()

fecha_dato               False
ncodpers                 False
ind_empleado             False
pais_residencia          False
sexo                     False
age                      False
fecha_alta               False
ind_nuevo                False
antiguedad               False
indrel                   False
indrel_1mes              False
tiprel_1mes              False
indresi                  False
indext                   False
canal_entrada            False
indfall                  False
nomprov                  False
ind_actividad_cliente    False
renta                    False
segmento                 False
ind_ahor_fin_ult1        False
ind_aval_fin_ult1        False
ind_cco_fin_ult1         False
ind_cder_fin_ult1        False
ind_cno_fin_ult1         False
ind_ctju_fin_ult1        False
ind_ctma_fin_ult1        False
ind_ctop_fin_ult1        False
ind_ctpp_fin_ult1        False
ind_deco_fin_ult1        False
ind_deme_fin_ult1        False
ind_dela_fin_ult1        False
ind_ecue

In [15]:
df.count() # number of instances

fecha_dato               81712
ncodpers                 81712
ind_empleado             81712
pais_residencia          81712
sexo                     81712
age                      81712
fecha_alta               81712
ind_nuevo                81712
antiguedad               81712
indrel                   81712
indrel_1mes              81712
tiprel_1mes              81712
indresi                  81712
indext                   81712
canal_entrada            81712
indfall                  81712
nomprov                  81712
ind_actividad_cliente    81712
renta                    81712
segmento                 81712
ind_ahor_fin_ult1        81712
ind_aval_fin_ult1        81712
ind_cco_fin_ult1         81712
ind_cder_fin_ult1        81712
ind_cno_fin_ult1         81712
ind_ctju_fin_ult1        81712
ind_ctma_fin_ult1        81712
ind_ctop_fin_ult1        81712
ind_ctpp_fin_ult1        81712
ind_deco_fin_ult1        81712
ind_deme_fin_ult1        81712
ind_dela_fin_ult1        81712
ind_ecue

## 2. Machine learning pipeline Implementation
Implement a machine learning pipeline in Spark, including feature extractors, transformers, and/or selectors. Test that your pipeline it is correctly implemented and explain your choice of processing steps, learning algorithms, and parameter settings.

In [16]:
# remove any previous spark session and check df file type
spark.stop()
type(df)

pandas.core.frame.DataFrame

In [17]:
# Create Spark SQL dataframe 
## IMPORTANT!! - this cell usually takes time due to data volume!!!
## IMPORTANT!! - Only run this cell once! (to run it again, you need to restart the kernel)

from pyspark.sql import SQLContext
sc = SparkContext()
sqlCtx = SQLContext(sc) #print(sc)
df_spark = sqlCtx.createDataFrame(df)
type(df_spark)


pyspark.sql.dataframe.DataFrame

In [18]:
# define datatypes in dataframe

df_spark = df_spark.select(df_spark.fecha_dato.cast("date"),
                                   df_spark.ncodpers.cast("float"),
                                   df_spark.ind_empleado.cast("string"),
                                   df_spark.pais_residencia.cast("string"),
                                   df_spark.sexo.cast("string"),
                                   df_spark.age.cast("float"),
                                   df_spark.fecha_alta.cast("date"),
                                   df_spark.ind_nuevo.cast("float"),
                                   df_spark.antiguedad.cast("float"),
                                   df_spark.indrel.cast("float"),
                                   df_spark.indrel_1mes.cast("float"),
                                   df_spark.tiprel_1mes.cast("string"),
                                   df_spark.indresi.cast("string"),
                                   df_spark.indext.cast("string"),
                                   df_spark.canal_entrada.cast("string"),
                                   df_spark.indfall.cast("string"),
                                   df_spark.nomprov.cast("string"),
                                   df_spark.ind_actividad_cliente.cast("float"),
                                   df_spark.renta.cast("float"),
                                   df_spark.segmento.cast("string"),
                                   df_spark.ind_ahor_fin_ult1.cast("float"),
                                   df_spark.ind_aval_fin_ult1.cast("float"),
                                   df_spark.ind_cco_fin_ult1.cast("float"),
                                   df_spark.ind_cder_fin_ult1.cast("float"),
                                   df_spark.ind_cno_fin_ult1.cast("float"),
                                   df_spark.ind_ctju_fin_ult1.cast("float"),
                                   df_spark.ind_ctma_fin_ult1.cast("float"),
                                   df_spark.ind_ctop_fin_ult1.cast("float"),
                                   df_spark.ind_ctpp_fin_ult1.cast("float"),
                                   df_spark.ind_deco_fin_ult1.cast("float"),
                                   df_spark.ind_deme_fin_ult1.cast("float"),
                                   df_spark.ind_dela_fin_ult1.cast("float"),
                                   df_spark.ind_ecue_fin_ult1.cast("float"),
                                   df_spark.ind_fond_fin_ult1.cast("float"),
                                   df_spark.ind_hip_fin_ult1.cast("float"),
                                   df_spark.ind_plan_fin_ult1.cast("float"),
                                   df_spark.ind_pres_fin_ult1.cast("float"),
                                   df_spark.ind_reca_fin_ult1.cast("float"),
                                   df_spark.ind_tjcr_fin_ult1.cast("float"),
                                   df_spark.ind_valo_fin_ult1.cast("float"),
                                   df_spark.ind_viv_fin_ult1.cast("float"),
                                   df_spark.ind_nomina_ult1.cast("float"),
                                   df_spark.ind_nom_pens_ult1.cast("float"),
                                   df_spark.ind_recibo_ult1.cast("float"))


In [19]:
df_spark.printSchema()

root
 |-- fecha_dato: date (nullable = true)
 |-- ncodpers: float (nullable = true)
 |-- ind_empleado: string (nullable = true)
 |-- pais_residencia: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- age: float (nullable = true)
 |-- fecha_alta: date (nullable = true)
 |-- ind_nuevo: float (nullable = true)
 |-- antiguedad: float (nullable = true)
 |-- indrel: float (nullable = true)
 |-- indrel_1mes: float (nullable = true)
 |-- tiprel_1mes: string (nullable = true)
 |-- indresi: string (nullable = true)
 |-- indext: string (nullable = true)
 |-- canal_entrada: string (nullable = true)
 |-- indfall: string (nullable = true)
 |-- nomprov: string (nullable = true)
 |-- ind_actividad_cliente: float (nullable = true)
 |-- renta: float (nullable = true)
 |-- segmento: string (nullable = true)
 |-- ind_ahor_fin_ult1: float (nullable = true)
 |-- ind_aval_fin_ult1: float (nullable = true)
 |-- ind_cco_fin_ult1: float (nullable = true)
 |-- ind_cder_fin_ult1: float (nullable =

In [20]:
# code modified from Spark documentation at:
# https://spark.apache.org/docs/2.1.0/ml-classification-regression.html#random-forest-classifier
# and DataBricks at:
# https://docs.databricks.com/spark/latest/mllib/binary-classification-mllib-pipelines.html

# imports dependencies for Random Forest pipeline
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer, OneHotEncoder, StringIndexer, VectorAssembler


# IMPORTANT - Define target label (for prediction) from target features
labels = "ind_ctju_fin_ult1"

# stages in the Pipeline
stages = []
    
# define variables; categorical, countinuous and target features

numericCols = ["age","antiguedad","renta"]

categoricalColumns = ["ind_empleado","pais_residencia","sexo","ind_nuevo","indrel", 
                      "indrel_1mes","tiprel_1mes", "indresi", "indext", "canal_entrada","nomprov", 
                      "ind_actividad_cliente","segmento"]

targetsColumns = ["ind_ahor_fin_ult1", "ind_aval_fin_ult1",
                        "ind_cco_fin_ult1", "ind_cder_fin_ult1", "ind_cno_fin_ult1",
                        "ind_ctma_fin_ult1", "ind_ctop_fin_ult1",
                        "ind_ctpp_fin_ult1", "ind_deco_fin_ult1", "ind_deme_fin_ult1", 
                        "ind_dela_fin_ult1", "ind_ecue_fin_ult1", "ind_fond_fin_ult1",
                        "ind_hip_fin_ult1", "ind_plan_fin_ult1", "ind_pres_fin_ult1",
                        "ind_reca_fin_ult1", "ind_tjcr_fin_ult1", "ind_valo_fin_ult1", 
                        "ind_viv_fin_ult1", "ind_nomina_ult1", "ind_nom_pens_ult1","ind_recibo_ult1"]


In [21]:
# Use OneHotEncoder to convert categorical variables into binary SparseVectors
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol,
                                  outputCol = categoricalCol + "Index") # Category Indexing with StringIndexer
    stages += [stringIndexer]  # Add stages to the pipeline

In [22]:
# define categorical index columns 
categoricalColumnsIDX = ["ind_empleadoIndex","pais_residenciaIndex","sexoIndex",
                         "ind_nuevoIndex","indrelIndex","indrel_1mesIndex",
                         "tiprel_1mesIndex","indresiIndex","indextIndex", 
                         "canal_entradaIndex","nomprovIndex","ind_actividad_clienteIndex","segmentoIndex"]

In [23]:
# Convert label into label indices using the StringIndexer
label_stringIdx = StringIndexer(inputCol = labels,
                                outputCol = "label")
stages += [label_stringIdx]

In [24]:
# Transform all features into a vector using VectorAssembler
assemblerInputs = categoricalColumnsIDX + numericCols + targetsColumns
assembler = VectorAssembler(inputCols = assemblerInputs,
                            outputCol = "features")
stages += [assembler]  # Add stage to the pipeline


In [25]:
prePipeline = Pipeline(stages = stages)
pipelineModel = prePipeline.fit(df_spark)

dataset = pipelineModel.transform(df_spark)

In [26]:
# Train a RandomForest model.
rf = RandomForestClassifier(labelCol = "label", 
                            featuresCol = "features", 
                            numTrees = 100,                 #  Number of trees in the random forest
                            impurity = 'entropy',            # Criterion used for information gain calculation
                            featureSubsetStrategy = "auto",
                            predictionCol = "prediction",
                            maxDepth = 5, 
                            maxBins = 50, 
                            minInstancesPerNode = 2) 
                            #minInfoGain=0.0, 
                            #subsamplingRate=1.0)


## Section C) Evaluation of Performance and testing (20%)
Evaluate the performance of your pipeline using training and test set (don’t use CV but pyspark.ml.tuning.TrainValidationSplit).

### 3.1. Evaluate performance of machine learning pipeline on training data and test data.

In [27]:
# imports dependencies
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [28]:
# Split data into training set and testing set
[trainData, testData] = dataset.randomSplit([0.8, 0.2], seed = 100)

In [29]:
# evaluation of model performance
evaluator = MulticlassClassificationEvaluator(labelCol = "label", 
                                              predictionCol = "prediction", 
                                              metricName = "accuracy")
# random forest parameters
paramGrid = ParamGridBuilder().addGrid(rf.numTrees, [100]).build()

# cross-validation of model performance during grid-search 
# Method: pyspark.ml.tuning.TrainValidationSplit
crossval = TrainValidationSplit(estimator = rf,
                                estimatorParamMaps = paramGrid,
                                evaluator = evaluator,
                                trainRatio = 0.9)

# Run cross-validation, and choose the best set of parameters.
print('starting cross-validation')
cvModel = crossval.fit(trainData)  # This takes time!
print('finished cross-validation')

starting cross-validation
finished cross-validation


In [30]:
# Make predictions for test set and compute test error
predictions = cvModel.transform(trainData)
train_accuracy = evaluator.evaluate(predictions)
print("Training Accuracy = %g" % (train_accuracy))
print("Training Error = %g" % (1.0 - train_accuracy))

Training Accuracy = 0.999067
Training Error = 0.00093345


In [31]:
# Make predictions for test set and compute test error
predictions = cvModel.transform(testData)
test_accuracy = evaluator.evaluate(predictions)
print("Test Accuracy = %g" % (test_accuracy))
print("Test Error = %g" % (1.0 - test_accuracy))

Test Accuracy = 0.998105
Test Error = 0.00189452


## Section D) Implement a parameter grid - Model fine-tuning (35%) 

### Note: This section takes time to compute!

Implement a parameter grid (using pyspark.ml.tuning.ParamGridBuilder[source]), varying at least one feature preprocessing step, one machine learning parameter, and the training set size. Document the training and test performance and the time taken for training and testing. Comment on your findings.

### 4.1. Evaluate model performance using a subset of preprocessing variables
#### No numeric predictors used, relaunch pipeline with this new preprocessing structure

In [32]:
# New preprocessing stage, without numeric predictors
new_stages = []

# remove preprocessing numeric predictors by including an empty vector
New_numericCols = [] # empty numeric predictors

# Add Newstages to the pipeline
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol,
                                  outputCol = categoricalCol + "Index")
    new_stages += [stringIndexer]  # Add stages to the pipeline

new_stages += [label_stringIdx]

# empty vector is inserted here
new_assemblerInputs = categoricalColumnsIDX + New_numericCols + targetsColumns
new_assembler = VectorAssembler(inputCols = new_assemblerInputs, outputCol = "features")

new_stages += [new_assembler]


In [33]:
# Creating new pipeline
from pyspark.ml import Pipeline

new_prePipeline = Pipeline(stages = new_stages)
new_pipelineModel = new_prePipeline.fit(df_spark)

new_dataset = new_pipelineModel.transform(df_spark)

In [34]:
[new_trainData, new_testData] = dataset.randomSplit([0.8, 0.2], seed = 100)

new_cvModel = crossval.fit(new_trainData)  # This takes time!

# Results:

new_predictions = cvModel.transform(new_trainData)
new_train_accuracy = evaluator.evaluate(new_predictions)
print("New Training Accuracy = %g" % (new_train_accuracy))
print("New Training Error = %g" % (1.0 - new_train_accuracy))

new_test_predictions = cvModel.transform(new_testData)
new_test_accuracy = evaluator.evaluate(new_test_predictions)
print("New Test Accuracy = %g" % (new_test_accuracy))
print("New Test Error = %g" % (1.0 - new_test_accuracy))

Training Accuracy = 0.999067
Training Error = 0.00093345
Test Accuracy = 0.998105
Test Error = 0.00189452


### 4.2. Training set size evaluation

In [35]:
print('Training set size evaluation')

%time

# size of different training set to be evaluated, and split of training set
sizes = [0.5, 0.1, 0.05, 0.01, 0.001]
data = trainData.randomSplit(sizes, seed = 100)

print('\n\n=== training set of size 100%, wait please')
cvModel = crossval.fit(trainData)
predictions = cvModel.transform(trainData)
accuracy = evaluator.evaluate(predictions)
print("Classification Error = %g" % (1.0 - accuracy))


Training set size evaluation
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 7.15 µs


=== training set of size 100%, wait please
Classification Error = 0.00093345


In [ ]:
%time

i = 0
for split in data:
    print('\n\n=== training set of size reduced to {}%, wait please'.format(sizes[i]*100))
    cvModel = crossval.fit(split)
    predictions = cvModel.transform(split)
    accuracy = evaluator.evaluate(predictions)
    print("Classification Error = %g" % (1.0 - accuracy))
    i+=1

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 10.5 µs


=== training set of size reduced to 50.0%, wait please
Classification Error = 0.00301217


=== training set of size reduced to 10.0%, wait please
Classification Error = 0.00729853


=== training set of size reduced to 5.0%, wait please
Classification Error = 0.00140168


=== training set of size reduced to 1.0%, wait please
Classification Error = 0.00433839


=== training set of size reduced to 0.1%, wait please
Classification Error = 0.00980392


### 4.3. Machine Learning Model Hyperparameter search

In [ ]:
# Define hyperparameters and their values to search and evaluate
%time

paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [10,50,100]) \
    .addGrid(rf.minInstancesPerNode, [1,3,5]) \
    .addGrid(rf.maxDepth, [2,5,8]).build()

# cross-validation of model performance during grid-search 
crossval = TrainValidationSplit(estimator = rf,
                                estimatorParamMaps = paramGrid,
                                evaluator = evaluator,
                                trainRatio = 0.9)

# Run cross-validation, and choose the best set of parameters.
print('starting Hyperparameter Grid Search with cross-validation')
cvModel = crossval.fit(trainData)
print('Grid Search with cross-validation has finished')

# pick best model
rfModel = cvModel.bestModel
print (rfModel)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 9.3 µs
starting Hyperparameter Grid Search with cross-validation


In [ ]:
# Make predictions for test set and compute test error
predictions = rfModel.transform(testData)
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

## Findings and conclusions:

As expected, Random Forest perform very well over this data due to its abstract nature. The average performance is practically perfect when executing the pipeline using Test Data (XXX Accuracy). The level of abstraction and flexibility is so high, that even when some predictors are removed the accuracy remains its levels, as seen in section 4.1

Furthermore, the different sizes of the dataset studied show that bigger sizes provide better results, because the number of cases (rows) seen by the algorithm increases the flexibility and the performance of the algorithm. Besides, bigger sizes avoid randomness. As seen in section 4.2, the accuracy varies widely depending on the luck. If the sample has good example cases the acuracy would be high, if you are unlucky and the cases are not representative the accuracy will be low. This is why the size in the sample is so important.

Moreover, as backed in theory a lot of small trees (low depth) provided good results with less computational cost. Because depth trees usually provides overfitting. This is studied in section 4.3

To sum up, Random Forest is a very good approach to this problem because of the nature of the data, large and complex. The good results obtained is the main proof.